# Basic Examples for Model with Metadata


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-seldon" modified.


## 1. Directly define in model

In [3]:
%%writefile models/init-metadata/Model.py

import logging
import json
import os


class Model:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features

    def init_metadata(self):
        logging.info("metadata method  called")

        meta = {
            "name": "my-model-name",
            "versions": ["my-model-version-01"],
            "platform": "seldon",
            "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1, 5]}],
            "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}],
        }

        return meta

Overwriting models/init-metadata/Model.py


In [17]:
%%writefile model-metadata/init-metadata-grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metadata:0.3
          name: my-model
          env:
          - name: API_TYPE
            value: GRPC            
          - name: SELDON_LOG_LEVEL
            value: DEBUG
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/init-metadata-grpc.yaml


In [19]:
%%bash

kubectl apply -f model-metadata/init-metadata-grpc.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model configured
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [22]:
%%bash
cd ../../../executor/proto && grpcurl \
    -d '{"name": "my-model"}' \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Metadata

{
  "name": "my-model-name",
  "versions": [
    "my-model-version-01"
  ],
  "platform": "seldon",
  "inputs": [
    {
      "name": "input",
      "datatype": "BYTES",
      "shape": [
        "1",
        "5"
      ]
    }
  ],
  "outputs": [
    {
      "name": "output",
      "datatype": "BYTES",
      "shape": [
        "1"
      ]
    }
  ]
}


In [33]:
%%bash
cd ../../../executor/proto && grpcurl \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/GraphMetadata

{
  "name": "example",
  "models": {
    "my-model": {
      "name": "my-model-name",
      "versions": [
        "my-model-version-01"
      ],
      "platform": "seldon",
      "inputs": [
        {
          "name": "input",
          "datatype": "BYTES",
          "shape": [
            "1",
            "5"
          ]
        }
      ],
      "outputs": [
        {
          "name": "output",
          "datatype": "BYTES",
          "shape": [
            "1"
          ]
        }
      ]
    }
  },
  "inputs": [
    {
      "name": "input",
      "datatype": "BYTES",
      "shape": [
        "1",
        "5"
      ]
    }
  ],
  "outputs": [
    {
      "name": "output",
      "datatype": "BYTES",
      "shape": [
        "1"
      ]
    }
  ]
}


In [35]:
!kubectl delete -f model-metadata/init-metadata.yaml

Error from server (NotFound): error when deleting "model-metadata/init-metadata.yaml": seldondeployments.machinelearning.seldon.io "seldon-model" not found


## 2. Via environmental variable

In [37]:
%%writefile model-metadata/environ-metadata-grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.3
          name: my-model
          env:
          - name: API_TYPE
            value: GRPC                
          - name: SELDON_LOG_LEVEL
            value: DEBUG               
          - name: MODEL_METADATA
            value: |
              ---
              name: my-model-name
              versions: [ my-model-version-01 ]
              platform: seldon
              inputs:
              - datatype: BYTES
                name: input
                shape: [ 1, 4 ]
              outputs:
              - datatype: BYTES
                name: output
                shape: [ 3 ]
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Writing model-metadata/environ-metadata-grpc.yaml


In [45]:
%%bash

kubectl apply -f model-metadata/environ-metadata-grpc.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model configured
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 1 old replicas are pending termination...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [46]:
%%bash
cd ../../../executor/proto && grpcurl \
    -d '{"name": "my-model"}' \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Metadata

{
  "name": "my-model-name",
  "versions": [
    "my-model-version-01"
  ],
  "platform": "seldon",
  "inputs": [
    {
      "name": "input",
      "datatype": "BYTES",
      "shape": [
        "1",
        "4"
      ]
    }
  ],
  "outputs": [
    {
      "name": "output",
      "datatype": "BYTES",
      "shape": [
        "3"
      ]
    }
  ]
}


In [47]:
%%bash
cd ../../../executor/proto && grpcurl \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/GraphMetadata

{
  "name": "example",
  "models": {
    "my-model": {
      "name": "my-model-name",
      "versions": [
        "my-model-version-01"
      ],
      "platform": "seldon",
      "inputs": [
        {
          "name": "input",
          "datatype": "BYTES",
          "shape": [
            "1",
            "4"
          ]
        }
      ],
      "outputs": [
        {
          "name": "output",
          "datatype": "BYTES",
          "shape": [
            "3"
          ]
        }
      ]
    }
  },
  "inputs": [
    {
      "name": "input",
      "datatype": "BYTES",
      "shape": [
        "1",
        "4"
      ]
    }
  ],
  "outputs": [
    {
      "name": "output",
      "datatype": "BYTES",
      "shape": [
        "3"
      ]
    }
  ]
}


In [48]:
!kubectl delete -f model-metadata/environ-metadata.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


## 3. Via environmental variable (V1 format)

This illustrates how to use v1 format to define metadata.

This example uses environmental variable but this would also be valid for `init_metadata` approach.

In [57]:
%%writefile model-metadata/environ-metadata-v1-grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.3
          name: my-model
          env:
          - name: API_TYPE
            value: GRPC              
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              ---
              name: my-model-name
              versions: [ my-model-version-01 ]
              platform: seldon
              inputs:
              - messagetype: array
                shape: [ 2, 2 ]
              outputs:
              - messagetype: array
                shape: [ 1 ]
    graph:
      children: []
      endpoint:
        type: REST
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/environ-metadata-v1-grpc.yaml


In [58]:
%%bash

kubectl apply -f model-metadata/environ-metadata-v1-grpc.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model configured
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 1 old replicas are pending termination...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [60]:
%%bash
cd ../../../executor/proto && grpcurl \
    -d '{"name": "my-model"}' \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Metadata

{
  "name": "my-model-name",
  "versions": [
    "my-model-version-01"
  ],
  "platform": "seldon",
  "inputs": [
    {
      "shape": [
        "2",
        "2"
      ],
      "messagetype": "array"
    }
  ],
  "outputs": [
    {
      "shape": [
        "1"
      ],
      "messagetype": "array"
    }
  ]
}


In [61]:
%%bash
cd ../../../executor/proto && grpcurl \
    -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/GraphMetadata

{
  "name": "example",
  "models": {
    "my-model": {
      "name": "my-model-name",
      "versions": [
        "my-model-version-01"
      ],
      "platform": "seldon",
      "inputs": [
        {
          "shape": [
            "2",
            "2"
          ],
          "messagetype": "array"
        }
      ],
      "outputs": [
        {
          "shape": [
            "1"
          ],
          "messagetype": "array"
        }
      ]
    }
  },
  "inputs": [
    {
      "shape": [
        "2",
        "2"
      ],
      "messagetype": "array"
    }
  ],
  "outputs": [
    {
      "shape": [
        "1"
      ],
      "messagetype": "array"
    }
  ]
}


In [62]:
!kubectl delete -f model-metadata/environ-metadata.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
